# Pandas Best Practices - df.pipe() method chaining

[![Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/dylanhogg/jupyter-experiments/blob/master/notebooks/best-practices/pandas-pipe-method.ipynb)    


# References
https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.pipe.html  
https://calmcode.io/pandas-pipe/introduction.html  
https://github.com/koaning/calm-notebooks  
https://github.com/wesm/pydata-book  
https://tomaugspurger.github.io/method-chaining.html  
https://github.com/Delgan/loguru  


# Dependencies

In [ ]:
%pip install loguru -q  # Logging services; https://github.com/Delgan/loguru

In [ ]:
import numpy as np
import pandas as pd
from datetime import datetime
from functools import wraps
from IPython.display import display, HTML

In [ ]:
from loguru import logger

In [ ]:
pd.options.display.max_columns = None
pd.options.display.max_rows = 100

## Load input data (popular python libraries)

In [ ]:
df_raw = pd.read_json("https://www.awesomepython.org/github_data.json", orient="table")

In [ ]:
df_raw[0:2]

## Decorator helpers

In [ ]:
def log_pipeline_step(func):
    @wraps(func)
    def wrapper(*args, **kwargs) -> pd.DataFrame:
        input_shape = args[0].shape
        logger.info(f"{func.__name__}")
        tic = datetime.now()
        df_result = func(*args, **kwargs)
        output_shape = df_result.shape
        logger.info(f" ╰╴{func.__name__} took {datetime.now() - tic}s in: {input_shape} out: {output_shape} diff: ({output_shape[0] - input_shape[0]}, {output_shape[1] - input_shape[1]})")
        return df_result
    return wrapper

def log_columns(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        df_result = func(*args, **kwargs)
        logger.info(f"{func.__name__} cols ({len(df_result.columns)}): [{', '.join(list(df_result.columns))}]")
        return df_result
    return wrapper

## Generic pipe functions

In [ ]:
@log_pipeline_step
@log_columns
def start_pipeline(dataf):
    return dataf.copy()

@log_columns
def end_pipeline(dataf):
    return dataf

@log_pipeline_step
def filter_rows(dataf: pd.DataFrame, column, min_value) -> pd.DataFrame:
    return (dataf[dataf[column] >= min_value])

@log_pipeline_step
def sort_values(dataf: pd.DataFrame, col_names, ascending=False) -> pd.DataFrame:
    return dataf.sort_values(by=col_names, ascending=ascending)

@log_pipeline_step
def move_col(dataf, col_name, index=0):
    cols = dataf.columns.tolist()
    cols.insert(index, cols.pop(cols.index(col_name)))
    return dataf.loc[:, cols]

@log_pipeline_step
def calc_sum(dataf: pd.DataFrame, index_name= "total") -> pd.DataFrame:
    def _numeric_sum(col):
        return col.sum() if np.issubdtype(col.dtype, np.number) else None
    dataf.loc[index_name] = dataf.apply(_numeric_sum, axis=0) # over columns
    return dataf

## Custom pipe functions

In [ ]:
@log_pipeline_step
def set_dtypes(dataf: pd.DataFrame) -> pd.DataFrame:
    return (dataf
            .assign(last_commit_date=lambda d: pd.to_datetime(d['last_commit_date']).dt.tz_localize(None))
            .assign(updated_at=lambda d: pd.to_datetime(d['updated_at']).dt.tz_localize(None))
            .assign(created_at=lambda d: pd.to_datetime(d['created_at']).dt.tz_localize(None))
           )

@log_pipeline_step
def remove_zero_age_weeks(dataf: pd.DataFrame) -> pd.DataFrame:
    dataf.loc[dataf["age_weeks"] == 0, "age_weeks"] = 1
    return dataf

@log_pipeline_step
def rename_columns(dataf: pd.DataFrame) -> pd.DataFrame:
    dataf.columns = dataf.columns.str.lstrip('_')
    dataf.columns = dataf.columns.str.replace(' ', '')
    return dataf

@log_pipeline_step
def calc_popularity_measure(dataf: pd.DataFrame, col_name = "popularity_measure") -> pd.DataFrame:
    def _calc_popularity_measure(row):
        return (row["stars"] + row["forks"] + row["watches"]) / row["age_weeks"]
    kwargs = {col_name: _calc_popularity_measure}
    return dataf.assign(**kwargs)

@log_pipeline_step
def expand_per_week_measures(dataf: pd.DataFrame) -> pd.DataFrame:
    def _calc_per_week_measures(row):
        return {
            "watches_per_week": row["watches"] / row["age_weeks"],
            "forks_per_week": row["forks"] / row["age_weeks"]
        }
    res = dataf.apply(_calc_per_week_measures, axis=1, result_type='expand')
    dataf[res.columns] = res
    return dataf

@log_pipeline_step
def expand_percentages(dataf: pd.DataFrame) -> pd.DataFrame:
    def _calc_percentages(row):
        return {
            "stars_percent": row["stars"] * 100 / dataf["stars"].sum(), 
            "watches_percent": row["watches"] * 100 / dataf["watches"].sum(), 
            "forks_percent": row["forks"] * 100 / dataf["forks"].sum()
        }
    res = dataf.apply(_calc_percentages, axis=1, result_type='expand')
    dataf[res.columns] = res
    return dataf

## Pipeline example

In [ ]:
df = (df_raw
      .pipe(start_pipeline)
      .pipe(rename_columns)
      .pipe(set_dtypes)
      .pipe(remove_zero_age_weeks)
      .pipe(filter_rows, column="stars", min_value=100)
      .pipe(expand_per_week_measures)
      .pipe(expand_percentages)
      .pipe(calc_popularity_measure, col_name="popularity_measure")
      .pipe(sort_values, col_names=["popularity_measure"])
      .pipe(calc_sum)
      .pipe(move_col, "popularity_measure", 0)
      .pipe(end_pipeline)
     )

In [ ]:
display(df)